<a href="https://colab.research.google.com/github/asegura4488/MetodosComputacionalesI2025/blob/main/Semana9/DescensoGraidienteSL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [31]:

def Plotter(e):

  fig = plt.figure(figsize=(6,3))
  ax = fig.add_subplot(121)
  ax1 = fig.add_subplot(122)

  ax.set_title('Epoch: {:.0f}'.format(e),fontsize=10)

  return fig,ax,ax1

In [40]:
G1=(lambda x,y,z: 3*x - y - z - 1., \
    lambda x,y,z: -x + 3*y + z - 3., \
    lambda x,y,z: 2*x + y + 4*z - 7.)

In [41]:
print(G1[0](1,1,1),G1[1](1,1,1),G1[2](1,1,1))

0.0 0.0 0.0


In [42]:
def GetJacobian(f,r,h=1e-3):

  n = r.shape[0]

  J = np.zeros((n,n))

  for i in range(n):
    for j in range(n):

      rf = r.copy()
      rb = r.copy()

      rf[j] = rf[j] + h
      rb[j] = rb[j] - h

      J[i,j] = (f[i](*rf) - f[i](*rb))/(2*h)

  return J

In [43]:
J = GetJacobian(G1,np.array([0.,0.5,0.4]))
J

array([[ 3., -1., -1.],
       [-1.,  3.,  1.],
       [ 2.,  1.,  4.]])

In [46]:
r0 = np.array([0.,0.5,0.])

In [47]:
J = GetJacobian(G1,r0)
J

array([[ 3., -1., -1.],
       [-1.,  3.,  1.],
       [ 2.,  1.,  4.]])

In [48]:
gamma = 0.05
r1 = r0 - gamma * J @ r0
r1

array([ 0.025,  0.425, -0.025])

In [49]:
r2 = r1 - gamma * GetJacobian(G1,r1) @ r1
r2

array([ 0.04125,  0.36375, -0.04375])